In [1]:
import pandas as pd
import random
import time
from datetime import date
import numpy as np

import mysql.connector as mysql
import datetime as dt
import matplotlib.pyplot as plt
from pandasql import sqldf
from dateutil.relativedelta import relativedelta
import itertools as it
from scipy.stats import beta

In [2]:
def match_schedule(list_of_teams):
    
    l1 = list(it.combinations(list_of_teams, 2))
    l2 = [list(i) for i in l1]
    l2[2].reverse()
    l2[3].reverse()
    l2[6].reverse()
    random.shuffle(l2) # pierwsza połowa ligi
    
    l3 = [list(reversed(i)) for i in l2] # druga połowa ligi
    
    return l2 + l3

In [139]:
match_schedule([1,2,3,4,5])

[[3, 5],
 [5, 2],
 [4, 5],
 [1, 2],
 [3, 4],
 [2, 4],
 [2, 3],
 [4, 1],
 [1, 3],
 [5, 1],
 [5, 3],
 [2, 5],
 [5, 4],
 [2, 1],
 [4, 3],
 [4, 2],
 [3, 2],
 [1, 4],
 [3, 1],
 [1, 5]]

In [58]:
my_connection = mysql.connect(
host="giniewicz.it",
user="student",
password="57u>3n7"
)

In [90]:
cursor = my_connection.cursor()

In [3]:
def generate_name(file):
    """
    Funkcja zwraca wylosowane imię ze 100 najbardziej znanych w Polsce imion. 
    """
    
    df = pd.read_csv(file)[1:100]
    n = df["IMIĘ_PIERWSZE"].sample(1)
    
    return n.to_string(index=False)

In [13]:
generate_name("IMIONA_ZENSKIE.csv")

'ANITA'

In [14]:
generate_name("IMIONA_MESKIE.csv")

'VOLODYMYR'

In [4]:
def generate_surname(file):
    """
    Funkcja zwraca wylosowane nazwisko ze 100 najbardziej znanych w Polsce nazwisk. 
    """
    
    df = pd.read_csv(file)[1:100]
    n = df["Nazwisko aktualne"].sample(1)
    
    return n.to_string(index=False)

In [7]:
generate_surname("NAZWISKA_ZENSKIE.csv")

'SZCZEPANIAK'

In [8]:
generate_surname("NAZWISKA_MESKIE.csv")

'PIOTROWSKI'

In [5]:
def str_time_prop(start, end, time_format, prop):
    """
    Zwraca datę pomiędzy dwiema ustalonymi (w odstępie takim jak argument (0,1))
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))

In [3]:
str_time_prop("01/01/1922", "01/01/1962", '%d/%m/%Y', 0.5)

'01/01/1942'

In [6]:
def random_date(start, end):
    """
    Losuje odstęp i wywołuje funkcje zwracającą date
    
    W ogólności funkcja za pomocą funkcji str_time_prop zwraca losową datę pomiędzy dwiema zadanymi 
    w argumentach danymi. Będzie do losowania JoinDate
    """
    prop=random.random()
    start = start.strftime('%d/%m/%Y')
    end = end.strftime('%d/%m/%Y')
    d = str_time_prop(start, end, '%d/%m/%Y', prop)
    return dt.datetime.strptime(d, '%d/%m/%Y')

In [18]:
random_date(dt.datetime(1979, 2, 18, 0, 0), dt.datetime(1992, 2, 18, 0, 0))

datetime.datetime(1989, 6, 9, 0, 0)

In [7]:
def generate_email(name, surname):
    
    """
    Funkcja zwraca email osoby 
    """
    
    number = int(random.random()*10000)
    mail = name.lower() + '.' + surname.lower() + str(number) + '@giniewicz.it.com'
    
    return mail

In [8]:
def generate_phone():
    
    """
    Funkcja zwraca numer osoby 
    """
    
    phone = random.randint(500000000, 900000000)
    return phone

In [15]:
generate_phone()

522679007

In [16]:
df = pd.read_csv("ADRESY_beta.csv", sep = ";")
df = df[["MIEJCOWOSC", "NAZWA_ULICY", "NUMER"]]

In [17]:
df.to_csv('ADRESY.csv', index = False)

In [9]:
def generate_address(file): 
    
    df = pd.read_csv(file)
    row = df.sample(1)
    
    city = row["MIEJCOWOSC"].to_string(index=False)
    street = row["NAZWA_ULICY"].to_string(index=False)
    number = row["NUMER"].to_string(index=False)
    
    address = street + " " + number + " " + city
    
    return address

In [19]:
generate_address("ADRESY.csv")

'ulica Potażowa 31 Warszawa'

In [10]:
def generate_gender():

    x = random.random()
    
    if 0 < x < 0.01:
        return "OTHER"
    elif 0.01 < x < 0.522:
        return "FEMALE"
    else:
        return "MALE"

In [28]:
generate_gender()

'MALE'

In [11]:
def generate_depart_date(company_start, current_date):
    
    """
    Funkcja zwraca datę odejścia pracownika lub gracza,
    musimy ustalić z jakimś prawdopodobieństwem ludzie odchodzą
    """
    
    x = random.random()
    
    if 0 < x < 0.1:
        date = random_date(company_start, current_date)
    else:
        date = "01/01/9999"
        date = dt.datetime.strptime(date, '%d/%m/%Y')
    
    
    return date

In [145]:
generate_depart_date("01/01/1922", "01/06/2022")

datetime.datetime(1979, 2, 18, 0, 0)

In [12]:
def generate_income(min_income, max_income):
    
    """
    Funckcja zwraca losowe dochody danego pracownika.
    """
    
    income = random.randint(min_income ,max_income)
    
    return income

In [13]:
def num_of_spectators(data, middle_of_season, _min = 0, _max = 200):
    
    """
    Funkcja zwraca liczbę osób na widowni w czasie meczu
    """
    
    delta = abs((middle_of_season - data).days) / 70 # różnica czasowa między środkiem sezonu, a naszym meczem /70, bo to polowa sezonu
    
    x = beta(2+3*delta, 5-3*delta).rvs()
    
    return round(x*200)  

In [151]:
spectators(dt.datetime(2022, 2, 18, 0, 0), dt.datetime(2022, 2, 20, 0, 0))

101

In [14]:
def score():
    
    score_a = round(beta(5,9).rvs()*20)
    score_b = round(beta(5,9).rvs()*20)
    
    if score_a == score_b:
        if np.random.binomial(1,0.5):
            score_a += 1
        else:
            score_b += 1
            
    return score_a, score_b
        

In [219]:
score()

(6, 11)

In [103]:
def effectiveness_in_match(score):
    
    e = round(beta(score + 1,3).rvs(),2)
    
    return float(e)

In [104]:
type(effectiveness_in_match(10))

float

In [145]:
def fill(cursor):
    
    staff_depart_dates = []
    staff_depart_id = []
    c = 0 
    def info_fill_staff():

        for i in range(1, 16):

            gender = generate_gender()
            if gender == "FEMALE":
                file1 = "IMIONA_ZENSKIE.csv"
                file2 = "NAZWISKA_ZENSKIE.csv"
            elif gender == "MALE":
                file1 = "IMIONA_MESKIE.csv"
                file2 = "NAZWISKA_MESKIE.csv"
            else:
                x = random.random()
                if 0 < x < 0.5:
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                else:
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"


            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2020, 9, 1)
            depart_date = generate_depart_date(dt.datetime(2020, 10, 1), dt.datetime.today())

            if depart_date != dt.datetime(9999, 1, 1, 0, 0):
                staff_depart_dates.append((i, depart_date))

            birth_date = random_date(dt.datetime(1958, 1, 1), dt.datetime(2002, 1, 1))
            email = generate_email(first_name, last_name)
            phone = generate_phone()

            qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (i, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
            
            
        for i in range(1, len(staff_depart_dates)+1):
            
            gender = generate_gender()
            if gender == "FEMALE":
                file1 = "IMIONA_ZENSKIE.csv"
                file2 = "NAZWISKA_ZENSKIE.csv"
            elif gender == "MALE":
                file1 = "IMIONA_MESKIE.csv"
                file2 = "NAZWISKA_MESKIE.csv"
            else:
                x = random.random()
                if 0 < x < 0.5:
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                else:
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"


            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            birth_date = random_date(dt.datetime(1958, 1, 1), dt.datetime.today())
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            join_date = staff_depart_dates[i-1][1]
            depart_date = dt.datetime(9999, 1, 1, 0, 0)
            
            qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (i+15, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
        
        global num_of_staff
        num_of_staff = len(staff_depart_dates) + 15
            
        for i in staff_depart_dates:
            staff_depart_id.append(i[0])
            
    def info_fill_players():
        
        for i in range(25):
            
            gender = "FEMALE"
            file1 = "IMIONA_ZENSKIE.csv"
            file2 = "NAZWISKA_ZENSKIE.csv"
            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2020, 10, 1) # miesiąc po otwarciu firmy
            depart_date = dt.datetime(9999, 9, 9) # na pałe
            birth_date = random_date(dt.datetime(1954, 1, 1), dt.datetime(1962, 1, 1)) # poniżej 70 hihi
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            
            qr = ('INSERT INTO team9.Info ' 
                  '(FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
        for i in range(25):
            
            gender = "FEMALE"
            file1 = "IMIONA_ZENSKIE.csv"
            file2 = "NAZWISKA_ZENSKIE.csv"
            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2020, 10, 1) # miesiąc po otwarciu firmy
            depart_date = dt.datetime(9999, 9, 9) # na pałe
            birth_date = random_date(dt.datetime(1932, 1, 1), dt.datetime(1949, 1, 1)) # powyżej 70, ale ponizej 90 hihi
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            
            qr = ('INSERT INTO team9.Info ' 
                  '(FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
        for i in range(25):
            
            gender = "MALE"
            file1 = "IMIONA_MESKIE.csv"
            file2 = "NAZWISKA_MESKIE.csv"
            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2020, 10, 1) # miesiąc po otwarciu firmy
            depart_date = dt.datetime(9999, 9, 9) # na pałe
            birth_date = random_date(dt.datetime(1954, 1, 1), dt.datetime(1962, 1, 1)) # poniżej 70 hihi
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            
            qr = ('INSERT INTO team9.Info ' 
                  '(FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
        for i in range(25):
            
            gender = "MALE"
            file1 = "IMIONA_MESKIE.csv"
            file2 = "NAZWISKA_MESKIE.csv"
            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2020, 10, 1) # miesiąc po otwarciu firmy
            depart_date = dt.datetime(9999, 9, 9) # na pałe
            birth_date = random_date(dt.datetime(1932, 1, 1), dt.datetime(1949, 1, 1)) # poniżej 70 hihi
            email = generate_email(first_name, last_name)
            phone = generate_phone()
            
            qr = ('INSERT INTO team9.Info ' 
                  '(FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
                 
            
    def staff_fill():
        
        positions = {1:"PREZES", 2:"WICEPREZES", 3:"KSIĘGOWY", 4:"WOŹNY", 13:"FIZJOTERAPEUTA", 14:"KONSERWATOR LODU",
                    15:"ZŁOTA RĄCZKA"}
        positions.update(dict.fromkeys([5, 6, 7, 8, 9, 10, 11, 12], "TRENER"))

        
        for i in range(1,16):
            qr = ('INSERT INTO team9.Staff ' 
                  '(StaffID, Position, InfoID, Active) ' 
                  'VALUES (%s, %s, %s, %s)')
            
            if i in staff_depart_id:
                active = False
            else:
                active = True
                
            cursor.execute(qr, (i, positions[i], i, active))
            
        for i in range(1, len(staff_depart_dates)+1):
            qr = ('INSERT INTO team9.Staff ' 
                  '(StaffID, Position, InfoID, Active) ' 
                  'VALUES (%s, %s, %s, %s)')
            
            active = True
            
            cursor.execute(qr, (i+15, positions[staff_depart_dates[i-1][0]], i+15, active))
            
    def salaries_fill():
        
        salaries = {"PREZES":15000.0, "WICEPREZES":10000.0, "KONSERWATOR LODU":4000.0}
        salaries.update(dict.fromkeys(["KSIĘGOWY","WOŹNY"], 3100.0))
        salaries.update(dict.fromkeys(["FIZJOTERAPEUTA", "ZŁOTA RĄCZKA"], 3500.0))
        salaries.update(dict.fromkeys(["TRENER"], 4000.0))
        
        qr = """SELECT COUNT(*) FROM team9.Staff"""
        cursor.execute(qr)
        k = cursor.fetchone()[0]
        
        
        salaries_list = []    
        for i in range(1,k+1):
            
            qr = ('SELECT JoinDate, DepartDate FROM team9.Info WHERE InfoID = %s ')
            cursor.execute(qr, [i])
            join_date, depart_date = cursor.fetchone()
            
            qr = ('SELECT Position FROM team9.Staff WHERE StaffID = %s ')
            cursor.execute(qr, [i])
            position = cursor.fetchone()[0]
            
            flag = 0 
            time_change = relativedelta(months = 1)
            month_later = join_date + time_change
            date = dt.date(month_later.year, month_later.month, 10) # pierwsza wypłata
            salary = salaries[position]
            while flag == 0:
                
                salaries_list.append((date, -salary, None, i, None, None, None)) # pierwsza wypłata normalna
                
                if np.random.binomial(1,0.05):
                    salary += 100 # premia
                elif np.random.binomial(1,0.02) and salary >= 3110: # minimalna krajowa
                    salary -= 100 # kara
                
            
                if date >= depart_date:
                    flag += 1
                date += time_change
                if date >= dt.date.today():
                    flag += 1
                    
        return salaries_list
                    
    def teams_fill():
        
        team_names = ["ŻABKI", "PSZCZÓŁKI", "KANGURY", "SŁONIKI", "ŁOSOSIE", "TYGRYSY", "PANTERY", "WILKI", 
                     "SZCZUPAKI", "KUNY", "MOTYLKI", "DELFINY", "KOZY", "IGUANY", "SARENKI", "KRÓWKI",
                     "KURY", "OWIECZKI", "RYSIE", "ZEBRY"]
        
        for i in range(5):
            
            team_name = team_names.pop(random.randint(0,len(team_names)-1))
            _type = "FEMALE"
            age_category = "JUNIORS"
            
            qr = ('INSERT INTO team9.Teams ' 
                      '(TeamName, Type, AgeCategory) ' 
                      'VALUES (%s, %s, %s)')
                
            cursor.execute(qr, (team_name, _type, age_category))
            
        for i in range(5):
            
            team_name = team_names.pop(random.randint(0,len(team_names)-1))
            _type = "FEMALE"
            age_category = "SENIORS"
            
            qr = ('INSERT INTO team9.Teams ' 
                      '(TeamName, Type, AgeCategory) ' 
                      'VALUES (%s, %s, %s)')
                
            cursor.execute(qr, (team_name, _type, age_category))
            
        for i in range(5):
            
            team_name = team_names.pop(random.randint(0,len(team_names)-1))
            _type = "MALE"
            age_category = "JUNIORS"
            
            qr = ('INSERT INTO team9.Teams ' 
                      '(TeamName, Type, AgeCategory) ' 
                      'VALUES (%s, %s, %s)')
                
            cursor.execute(qr, (team_name, _type, age_category))
            
        for i in range(5):
            
            team_name = team_names.pop(random.randint(0,len(team_names)-1))
            _type = "MALE"
            age_category = "SENIORS"
            
            qr = ('INSERT INTO team9.Teams ' 
                      '(TeamName, Type, AgeCategory) ' 
                      'VALUES (%s, %s, %s)')
                
            cursor.execute(qr, (team_name, _type, age_category))
            
    def players_fill():
        
        team_id = 1 
        players_list = []
        info_id = num_of_staff
        for i in range(1, 101):
                
            active = True
            info_id += 1
            
            players_list.append((team_id, active, info_id))
            
            if i%5 == 0:
                team_id += 1
                
        qr = ('INSERT INTO team9.Players ' 
                      '(TeamID, Active, InfoID) ' 
                      'VALUES (%s, %s, %s)')
                
        cursor.executemany(qr, players_list)
        
    def player_tuition_fill():
        
        qr = """SELECT COUNT(*) FROM team9.Players"""
        cursor.execute(qr)
        k = cursor.fetchone()[0] # liczba graczy - na później 
        
        
        tuition_list = []    
        for i in range(1,k+1):
            
            qr = ('SELECT JoinDate, DepartDate FROM team9.Info WHERE InfoID = %s ')
            cursor.execute(qr, [i + num_of_staff])
            join_date, depart_date = cursor.fetchone()
            
            flag = 0 
            time_change = relativedelta(months = 1)
            #month_later = join_date + time_change
            date = dt.date(join_date.year, join_date.month, 10) # pierwsza wpłata DO ZMIANY, JEŚLI ZMIENIMY PRZYCHODZENIE I ODCHODZENIE PLAYERS
            amount = 50
            while flag == 0:
                
                tuition_list.append((date, amount, i, None, None, None, None)) # pierwsza wypłata normalna
                
                if date >= depart_date:
                    flag += 1
                date += time_change
                if date >= dt.date.today():
                    flag += 1
                    
        return tuition_list
        
        
    def matches_fill():
        
        # 3 sezony, bo 2 w tył i 1 w przód
        seasons = {0:"2020/2021", 1:"2021/2022", 2:"2022/2023"}
        
        matches = []
        for n in range(3): # mamy 3 sezony 
            
            # plan meczy 
            female_juniors = match_schedule([1,2,3,4,5])
            female_seniors = match_schedule([6,7,8,9,10])
            male_juniors = match_schedule([11,12,13,14,15])
            male_seniors = match_schedule([16,17,18,19,20])
            
            season_start = dt.datetime(2020 + n, 11, 7 - n, 12, 0)
            season_end = dt.datetime(2021 + n, 3, 21 - n, 16, 0)
            middle = season_start + (season_end - season_start) / 2
            
            date = season_start
            
            schedules = {1:female_juniors, 2:female_seniors, 3:male_juniors, 0:male_seniors}
            
            k = 0
            for i in range(1, 81):
            
                
                if n != 2: # tylko dla przeszłych meczy
                    spectators = num_of_spectators(date, middle)
                    score_a, score_b = score()
                else:
                    spectators = None
                    score_a, score_b = None, None
                    
                season = seasons[n]
                team_a, team_b = schedules[i%4][k]
                
                matches.append((spectators, season, date, score_a, score_b, team_a, team_b))
                
                if i%4 == 0:
                    date += dt.timedelta(hours = 140)
                elif i%2 == 0:
                    date += dt.timedelta(hours = 20)
                else:
                    date += dt.timedelta(hours = 4)
                    
                if i%4 == 0:
                    k += 1
                
        qr = ('INSERT INTO team9.Matches ' 
                  '(Spectators, Season, Date, ScoreA, ScoreB, TeamA, TeamB) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s)')
        
        cursor.executemany(qr, matches)
        
    def player_match_performance_fill():
        
        positions_team_a = ["PIERWSZY", "DRUGI", "TRZECI", "CZWARTY", "REZERWOWY"]
        positions_team_b = ["PIERWSZY", "DRUGI", "TRZECI", "CZWARTY", "REZERWOWY"]

        match_performance = []
        
        qr = """SELECT ScoreA, ScoreB, TeamA, TeamB FROM team9.Matches"""
        cursor.execute(qr)
        match_info = cursor.fetchall()
        
        for i in range(1, 161):
            
                score_a, score_b, team_a, team_b = match_info[i - 1]
                
                random.shuffle(positions_team_a) # tasujemy 
                random.shuffle(positions_team_b) # tasujemy 
                
                for j in range(1, 6):
                    
                    player_id_a = int(team_a * 5 - 5 + j)
                    player_id_b = int(team_b * 5 - 5 + j)
                    
                    position_a = positions_team_a[j-1]
                    position_b = positions_team_b[j-1]
                    
                    if position_a == "REZERWOWY":
                        effectiveness_a = None
                    else:
                        effectiveness_a = effectiveness_in_match(score_a)
                        
                    if position_b == "REZERWOWY":
                        effectiveness_b = None
                    else:
                        effectiveness_b = effectiveness_in_match(score_b)
                        
                    match_performance.append((effectiveness_a, position_a, player_id_a, i))
                    match_performance.append((effectiveness_b, position_b, player_id_b, i))
                    
        qr = ('INSERT INTO team9.PlayerMatchPerformance ' 
              '(Effectiveness, Position, PlayerID, MatchID) ' 
              'VALUES (%s, %s, %s, %s)')

        cursor.executemany(qr, match_performance)
        
        
    def items_fill():
        
        qr = """SELECT TeamName FROM team9.Teams"""
        cursor.execute(qr)
        team_names = cursor.fetchall()
        
        items = []
        for i in range(len(team_names)):
            
            if i < 10:
                size = "UNIVERSAL FEMALE"
            else:
                size = "UNIVERSAL MALE"
                
            description = team_names[i][0] + " - TEAM UNIFORM"
            price = 359
            quantity = 5
            items.append((size, description, price, quantity)) 
            
        # buty
        shoes_sizes = {37:2}
        shoes_sizes.update(dict.fromkeys([35, 36, 43, 44, 45], 1))
        shoes_sizes.update(dict.fromkeys([38, 39, 40, 41, 42], 3))
        
        for i in range(35, 46):
            
            size = str(i)
            description = "CURLING SHOES"
            price = 359 
            quantity = shoes_sizes[i]
            items.append((size, description, price, quantity)) 
            
        # szczotki
            
        size = "UNIVERSAL"
        description = "BROOM"
        price = 259
        quantity = 30
        items.append((size, description, price, quantity)) 
        
        # kamienie
        
        size = "UNIVERSAL"
        description = "MATCH STONE"
        price = 2137 # papieżowa
        quantity = 16 
        items.append((size, description, price, quantity)) 
        
        description = "TRAINING STONE"
        price = 1069
        quantity = 32
        items.append((size, description, price, quantity)) 
        
        qr = ('INSERT INTO team9.Items ' 
              '(Size, Description, Price, Quantity) ' 
              'VALUES (%s, %s, %s, %s)')

        cursor.executemany(qr, items)
        
    def cash_flow_items():
        
        qr = """SELECT ItemID, Quantity, Price FROM team9.Items"""
        cursor.execute(qr)
        items_info = cursor.fetchall()
        
        items_list = []
        for i in range(len(items_info)):
            
            item_id = items_info[i][0]
            date = dt.datetime(2020, 9, 1)
            amount = items_info[i][1] * items_info[i][2]
            
            items_list.append((date, -amount, None, None, item_id, None, None))
            
        return items_list
    
    def outside_income_fill():
        
        outside_income = [("EUROPEAN UNION", 1000000.0, dt.datetime(2020, 9, 1), dt.datetime(2020, 9, 1)), 
                         ("CITY COUNCIL", 10000.0, dt.datetime(2020, 9, 1), dt.datetime(9999, 1, 1)),
                         ("TARCZYŃSKI", 10000.0, dt.datetime(2020, 9, 1), dt.datetime(9999, 1, 1)),
                         ("VOLTAREN", 15000.0, dt.datetime(2020, 11, 1), dt.datetime(2021, 3, 1)),
                         ("ZAKŁAD POGRZEBOWY AS BYTOM", 5000.0, dt.datetime(2021,1, 1), dt.datetime(2021, 12, 1)),
                         ("STOPERAN", 8000.0, dt.datetime(2021,11, 1), dt.datetime(2022, 3, 1)),
                         ("BIEDRONKA", 10000.0, dt.datetime(2022,1, 1), dt.datetime(2022, 12, 1)),]
        
        
        qr = ('INSERT INTO team9.OutsideIncome ' 
              '(SponsorName, Amount, StartDate, EndDate) ' 
              'VALUES (%s, %s, %s, %s)')

        cursor.executemany(qr, outside_income)
        
            
        
        

    # wywołujemy każdą z funkcji po kolei - wypełniającą pojedynczą tabelę
    info_fill_staff()
    staff_fill()
    salaries_fill()
    info_fill_players()
    teams_fill()
    players_fill()
    player_tuition_fill()
    matches_fill()
    player_match_performance_fill()
    items_fill()
    outside_income_fill()
    

In [146]:
my_connection = mysql.connect(
host="giniewicz.it",
user="team9",
password="te@mqP@ss"
)

In [147]:
cursor = my_connection.cursor()

In [148]:
fill(cursor)

In [149]:
my_connection.commit()
cursor.close()
my_connection.close()

In [93]:
def fill2(cursor):
    
    staff_depart_dates = []
    def info_fill_staff():
        
        for i in range(15): # mamy 15 pracownikow

            gender = generate_gender()
            if gender == "FEMALE":
                file1 = "IMIONA_ZENSKIE.csv"
                file2 = "NAZWISKA_ZENSKIE.csv"
            elif gender == "MALE":
                file1 = "IMIONA_MESKIE.csv"
                file2 = "NAZWISKA_MESKIE.csv"
            else:
                x = random.random()
                if 0 < x < 0.5:
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                else:
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"


            first_name = generate_name(file1)
            last_name = generate_surname(file2)
            address = generate_address("ADRESY.csv")
            join_date = dt.date(2021, 1, 1)
            depart_date = generate_depart_date("01/01/2021", "01/06/2022")
            birth_date = random_date("01/01/1958", "01/01/2004")
            email = generate_email(first_name, last_name)
            phone = generate_phone()

   
            j = 0
            while depart_date != dt.datetime(9999, 1, 1, 0, 0):
                staff_depart_dates.append((i+j, depart_date))

                gender = generate_gender()
                if gender == "FEMALE":
                    file1 = "IMIONA_ZENSKIE.csv"
                    file2 = "NAZWISKA_ZENSKIE.csv"
                elif gender == "MALE":
                    file1 = "IMIONA_MESKIE.csv"
                    file2 = "NAZWISKA_MESKIE.csv"
                else:
                    x = random.random()
                    if 0 < x < 0.5:
                        file1 = "IMIONA_ZENSKIE.csv"
                        file2 = "NAZWISKA_ZENSKIE.csv"
                    else:
                        file1 = "IMIONA_MESKIE.csv"
                        file2 = "NAZWISKA_MESKIE.csv"

                first_name = generate_name(file1)
                last_name = generate_surname(file2)
                address = generate_address("ADRESY.csv")
                join_date = depart_date
                depart_date = generate_depart_date(depart_date, "01/06/2022")
                birth_date = random_date("01/01/1958", "01/01/2004")
                email = generate_email(first_name, last_name)
                phone = generate_phone()
                
                qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
                cursor.execute(qr, (i+j, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
                j = j+1
                    

            qr = ('INSERT INTO team9.Info ' 
                  '(InfoID, FirstName, LastName, Address, JoinDate, DepartDate, BirthDate, Email, Phone, Gender) ' 
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
            cursor.execute(qr, (i+1, first_name, last_name, address, join_date, depart_date, birth_date, email, phone, gender))
            
        

In [ ]:
def fill_items(cursor):
    #wstępna tabela items i itemcosts. nie dawałem tutaj ciuchów, bo wsm nie pamietam na czym stanelo
    
    qr_itemCosts = ('INSERT INTO team9.ItemCosts ' 
          '(Date, Amount) ' 
          'VALUES (%s, %s)')
    
    qr_item = ('INSERT INTO team9.Items ' 
          '(Size, PaymentID, Type, Broken) ' 
          'VALUES (%s, %s, %s, %s)')
    
    #liczba poszczególnych itemów i ich cena. Odpowiednio: kamienie, szczotki, buty i ta maszyna do lodu
    items_price = {35:random.randint(5000,7000), 30:random.randint(200,400), 20:random.randint(500,700), 1:random.randint(150000,200000)}
    
    #dodajemy ceny. Wszystko kupujemy w pierwszy dzien dzialalnosci klubu
    for key, val in items_price.items():
        for i in range(key):
            cursor.execute(qr_itemCosts, (dt.date(2021, 1, 1), val))
    
    #dodajemy itemy i odpowiedni klucz do ceny. Sprzet moze byc zepsuty z prawdop. 5%
    for i in range(35):
        cursor.execute(qr_item, ('-', i+1, 'KAMIEN', np.random.binomial(1, 0.05)))
    for i in range(30):
        cursor.execute(qr_item, ('-', i+36, 'SZCZOTKA', np.random.binomial(1, 0.05)))
    for i in range(20):
        #losujemy numer buta
        cursor.execute(qr_item, (str(random.randint(33,45)), i+66, 'BUTY', np.random.binomial(1, 0.05)))
        
    #maszyna za ponad 150 koła nie może się zepsuć xd
    cursor.execute(qr_item, ('-', 86, 'ROLBA', 0))